In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
# import math 
import os

In [2]:
transactions_data_path="kaggle_data/transactions_train.csv"

In [3]:
transactions=pd.read_csv(transactions_data_path)
transactions['t_dat']=transactions['t_dat'].astype('datetime64[ns]')

In [4]:
article_price = (
    transactions
    .groupby('article_id')             # Group by products
    .agg({'price': ['mean','min','max'],'t_dat': ['min','max'],'article_id':['count']}) 
    .reset_index()
)

article_price.columns = ['_'.join(col) for col in article_price.columns.values]
article_price= article_price.rename({"article_id_": "article_id"}, axis=1) 
article_price.head(2)

,article_id,price_mean,price_min,price_max,t_dat_min,t_dat_max,article_id_count
0,108775015,0.008142,0.001339,0.009153,2018-09-20,2020-07-22,10841
1,108775044,0.008114,0.001424,0.008508,2018-09-20,2020-09-20,7250


In [5]:
article_price.dtypes

article_id                   int64
price_mean                 float64
price_min                  float64
price_max                  float64
t_dat_min           datetime64[ns]
t_dat_max           datetime64[ns]
article_id_count             int64
dtype: object

In [6]:
products_data_path="kaggle_data/articles.csv"

In [7]:
products=pd.read_csv(products_data_path)

In [8]:
enhanced_products=products.merge(article_price, how='outer',on='article_id')

enhanced_products=enhanced_products.assign(
    price_percentile=enhanced_products.groupby(['product_group_name','index_group_name'])['price_mean'].transform('rank', pct=True)
)   

enhanced_products.head(2)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,garment_group_no,garment_group_name,detail_desc,price_mean,price_min,price_max,t_dat_min,t_dat_max,article_id_count,price_percentile
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,1002,Jersey Basic,Jersey top with narrow shoulder straps.,0.008142,0.001339,0.009153,2018-09-20,2020-07-22,10841.0,0.029462
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,1002,Jersey Basic,Jersey top with narrow shoulder straps.,0.008114,0.001424,0.008508,2018-09-20,2020-09-20,7250.0,0.027742


In [9]:
enhanced_products['article_id_count']=enhanced_products['article_id_count'].fillna(0)

In [17]:
# enhanced_products.describe(include="number").T
# uncomment to check 

In [10]:
path='kaggle_data/'
enhanced_products.to_csv(os.path.join(path,r'enhanced_products.csv'),index=False)

In [19]:
# cd kaggle_data

/home/jupyter/h_and_m/kaggle_data


In [21]:
# rm articles.csv